In [45]:
#Import modules
import pandas as pd
import csv
import networkx as nx
import numpy as np
from tqdm import tqdm
import shutil
import os
from datetime import datetime
import math
from collections import Counter
from numpy.linalg import norm
from scipy.spatial import distance
from multiprocessing import Pool
import numpy as np
import scipy.spatial.distance as dist
import warnings


In [46]:
#Entropy function
def H(X):
    s=sum(X)
    h=0
    for i in X:
        if i!=0:
            p=float(i/s)
            h=h-(p)*math.log(p)
    return h

#JSD function
def Calculate_JSD(com,user_link,user_link_Entropy):
    XX=np.sum([1/len(com)*user_link[i,:] for i in com],axis=0)
    JSD=H(XX)
    for i in com:
        JSD=JSD-((float)(1/len(com))*user_link_Entropy[i])
    return JSD

In [47]:
#Cosine similarity function
def Cosine_similarity(user_link1,user_link2):
    if norm(user_link1)*norm(user_link2)==0:
        return 0
    return np.dot(user_link1,user_link2)/(norm(user_link1)*norm(user_link2))


#Sort list and count
def sort_list(data_key,data_non_key,data_non_key2):
    for i in range(len(data_key)):
        loc=i
        maximum = data_key[i]  
        for j in range(i+1,len(data_key)): 
            if data_key[j] >maximum:
                maximum =  data_key[j]
                loc=j

        tmp=data_key[i]
        data_key[i]=data_key[loc]
        data_key[loc]=tmp
        tmp=data_non_key[i]
        data_non_key[i]=data_non_key[loc]
        data_non_key[loc]=tmp
        tmp2=data_non_key2[i]
        data_non_key2[i]=data_non_key2[loc]
        data_non_key2[loc]=tmp2

In [48]:
#Recursivly remove once repeated entitis and users
def recursive_remove(Posts):
    Flag=True
    while (Flag):
        len1=len(Posts)
        grouped = Posts.groupby('Link')
        Once_shared=[]
        for group in grouped:
            uids=set()
            matrix=np.array(group[1][['UserID','Link']].values.tolist())
            uids=set(matrix[:,0].tolist())
            if len(uids)<=1:
                Once_shared.append(matrix[0][1])
        Posts =Posts[~Posts["Link"].isin(list(Once_shared))]
        Flag=False

        len2=len(Posts)
        if len1==len2:
            Flag=False
        print('len1={}\tlen2={}'.format(len1,len2))

        grouped = Posts.groupby('UserID')
        shared_one_link=[]
        for group in grouped:
            lids=set()
            matrix=np.array(group[1][['UserID','Link']].values.tolist())
            lids=set(matrix[:,1].tolist())
            if len(lids)<=1:
                shared_one_link.append(matrix[0][0])
        Posts =Posts[~Posts["UserID"].isin(list(shared_one_link))]
        len3=len(Posts)
        if len3==len2:
            Flag=False
        print('len2={}\tlen3={}'.format(len2,len3))
        
    print("The number of posts: ", len(Posts))
    print("The number of unique entities: ", Posts.UserID.nunique())
    print("The number of unique links: ", Posts.Link.nunique())

    return Posts


In [49]:
#Read posts 
import re
#===============================================================================================================================================================================================
def extract_hashtags(text: str) -> list[str]:
    "Extracts hashtags from text and returns a list."
    pattern = r"(#\w+)" #r"#[^\s!@#$%^&*()=+./,\[{\]};:'\"?><]+"
    return re.findall(pattern, text)
#Assign Numerical ID
def assign_numerical_ID(P,field,ID_field):
    unique_set = P[field].unique()
    entity_to_id = {entity: id for id, entity in enumerate(unique_set)}
    P[ID_field] = P[field].map(entity_to_id)
    return P,len(unique_set)
#===============================================================================================================================================================================================
def read_post(file_name,from_time,to_time):
    data = pd.read_csv(file_name, delimiter=',', skipinitialspace=True, engine='python',converters={"Post_ID":str, "Post_text":str,"User_ID":str,"Post_text":str})
    #data = pd.read_csv(file_name, delimiter=',', skipinitialspace=True, engine='c',converters={"Post_ID":str, "Post_text":str,"User_ID":str,"Post_text":str})
    data['Post_text'] = data['Post_text'].astype(str)

    # Get column indices
    text_column = data.columns.get_loc('Post_text')
    user_column = data.columns.get_loc('User_ID')
    time_column = data.columns.get_loc('Post_time')
    post_column = data.columns.get_loc('Post_ID')
    is_control_column = data.columns.get_loc('is_control')

    hashtag_data = []
    for row in data.itertuples(index=False):
        text = row[text_column]  # 'Post_text'
        hashtags = extract_hashtags(text.lower())

        # Ensure unique hashtags
        hashtags = set(hashtags)

        # Handle missing or problematic Post_time
        time_value = row[time_column]

        # Check if time_value is not None or NaN before processing
        if pd.isna(time_value) or not isinstance(time_value, str):
            continue  # Skip this row

        try:
            timestamp = (time_value.split('+')[0])
        except (ValueError, pd.errors.OutOfBoundsDatetime):
            continue  # Skip this row if there's an error in datetime conversion

        for hashtag in hashtags:
            hashtag_data.append({
                'UserID': row[user_column],  # 'User_ID'
                'Hashtag': hashtag,
                'Time':datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S'),
                #'Time': timestamp,  # Processed timestamp
                'PostID': row[post_column],  # 'Post_ID'
                'Is_Control': row[is_control_column]
            })

    hashtag_df = pd.DataFrame(hashtag_data)
    hashtag_df.columns = ['UserID', 'Link', 'PostDate', 'PostID', 'Is_Control']
    Posts = hashtag_df[['UserID', 'Link', 'PostDate', 'PostID', 'Is_Control']]

    Posts=Posts.loc[(Posts.PostDate>=from_time)&(Posts.PostDate<=to_time)].copy()

    Posts=recursive_remove(Posts)
    Posts,user_count=assign_numerical_ID(Posts,'UserID','Numeric_UID')
    Posts,link_count=assign_numerical_ID(Posts,'Link','Numeric_LID')

    return Posts[['UserID', 'Link', 'PostDate','PostID','Is_Control','Numeric_UID','Numeric_LID']], user_count,link_count



In [ ]:
#Read the dataset 
main_path_input =''#input file
main_path_output=''#outputfile

from_time=datetime.strptime('2017-06-22 00:00:00', '%Y-%m-%d %H:%M:%S')
to_time=datetime.strptime('2018-06-21 23:59:59', '%Y-%m-%d %H:%M:%S')



Posts,user_count,link_count=read_post(main_path_input,from_time,to_time)


len1=78962	len2=52760
len2=52760	len3=51607
The number of posts:  51607
The number of unique entities:  1684
The number of unique links:  4634


In [51]:
#Main 1-Multi-edge bipartit graph
def Multi_edge_graph(Posts):
        
    grouped = Posts.groupby('Link')
    max_time=0
    total_time=0
    counter=0
    for group in grouped:
        matrix=(list(group)[1])[['UserID','PostDate','Link','Numeric_UID','Numeric_LID']].values.tolist()
        if len(matrix)>0:
            matrix.sort(key= lambda c: (pd.to_datetime(c[1])),reverse=False)   
            t0=pd.to_datetime(matrix[0][1])
            tn=pd.to_datetime(matrix[len(matrix)-1][1])
            timedecay=(tn-t0).total_seconds()
            if timedecay>max_time:
                max_time=timedecay
            total_time+=timedecay
            counter+=1
    alpha=(math.log(10000)/(total_time/counter))

    UserID=[]
    Expt=[]
    Link=[]
    UserNID=[]
    LinkNID=[]
    for group in grouped:
        matrix=(list(group)[1])[['UserID','PostDate','Link','Numeric_UID','Numeric_LID']].values.tolist()
        matrix.sort(key= lambda c: (pd.to_datetime(c[1])),reverse=False)   
        t0=pd.to_datetime(matrix[0][1])

        for i in range(len(matrix)):
            ti=pd.to_datetime(matrix[i][1])
            td=(ti-t0).total_seconds()
            timedecay=math.exp(-alpha*td)
            if (timedecay>0.00001):
                UserID.append(matrix[i][0])
                timedecay=math.exp(-alpha*td)
                Expt.append(timedecay)
                Link.append(matrix[i][2])
                UserNID.append(matrix[i][3])
                LinkNID.append(matrix[i][4])

    
    df=pd.DataFrame()
    df['UserID']=UserID
    df['TimeDecay']=Expt
    df['Link']=Link
    df['Numeric_UID']=UserNID
    df['Numeric_LID']=LinkNID
    return df


MEBgraph=Multi_edge_graph(Posts)

In [52]:
#Main 2-Convert Muti-edge Bipartite graph into Single_edge_Bipartite graph
def Single_edge_graph_summation(Posts):
    grouped = Posts.groupby(['UserID','Link'])
    UserID=[]
    UserNID=[]
    Usage=[]
    Link=[]
    LinkNID=[]
    Number_link_used=[]

    counter_rep_list=[]
    for group in grouped:

        matrix=(list(group)[1])[['UserID','TimeDecay','Link','Numeric_UID','Numeric_LID']].values.tolist()
        p=0
        c=0
        if len(matrix)>1:
            counter_rep_list.append(len(matrix))  
        for i in range(len(matrix)):
            p+=matrix[i][1]
            c+=1

        UserID.append(matrix[i][0])
        Usage.append(p)
        Link.append(matrix[i][2])
        UserNID.append(matrix[i][3])
        LinkNID.append(matrix[i][4])
        Number_link_used.append(c)
    
    
    df=pd.DataFrame()
    df['UserID']=UserID
    df['Usage']=Usage
    df['Link']=Link
    df['Numeric_UID']=UserNID
    df['Numeric_LID']=LinkNID
    df['Number_link_used']=Number_link_used
    return df

SEBgraph=Single_edge_graph_summation(MEBgraph)

In [53]:
#Calculate initial coordination values
def Link_Usage_Behaviour_Matrix3(SEBgraph):
    user_behaviour_weight=[]
    user_behaviour=[]
    user_ID_behaviour=[]
    user_behaviour_on_link=[]
    min_repeatation=[]
    max_repeatition=[]
    
    grouped = SEBgraph.groupby('Numeric_LID')
    for group in grouped:
        UserIDs=[]
        beha=[]
        Number_link_used=[]
        matrix=(list(group)[1])[['Numeric_LID','Usage','Number_link_used','Numeric_UID','Link']].values.tolist()

        if len(matrix)>1:
            for i in range(len(matrix)):
                UserIDs.append(matrix[i][3])
                beha.append(matrix[i][1])
                Number_link_used.append(matrix[i][2])
            sort_list(beha,UserIDs,Number_link_used)
            max_vector=[]
            max_value=-1
            for pivot in range(0,len(matrix)):    
                h1=H(beha[:pivot]) 
                h2=H(beha[pivot:])
                if h1>h2:
                    if h1>max_value:
                        max_value=h1
                        max_vector=[beha[:pivot],beha[pivot:]]
                else:
                    if h2>max_value:
                        max_value=h2
                        max_vector=[beha[:pivot],beha[pivot:]]
           
            if len(max_vector[0])>0:
                vector=max_vector[0]
            else:
                vector=max_vector[1]


            if len(vector)==len(beha):
                if abs(H(beha)-H(beha[:-1]))<abs(np.std(beha)-np.std(beha[:-1])):
                    vector=vector[:-1]

            coordination_size=len(vector)

            if coordination_size>1:
                user_ID_behaviour.append(UserIDs[:coordination_size])
                user_behaviour.append(beha[:coordination_size])
                user_behaviour_on_link.append(matrix[0][0])
                user_behaviour_weight.append((H(vector)/math.log(coordination_size))*(np.sum(Number_link_used[:coordination_size])/len(Number_link_used[:coordination_size]))*(H(Number_link_used[:coordination_size])/math.log(coordination_size)))
                min_repeatation.append(np.min(Number_link_used[:coordination_size]))
                max_repeatition.append(np.max(Number_link_used[:coordination_size]))
    
    
    return user_behaviour,user_ID_behaviour,user_behaviour_on_link

users_behaviour,edges_users,coordinated_link=Link_Usage_Behaviour_Matrix3(SEBgraph)

In [54]:
#DTW function
def dp(dist_mat):

    N, M = dist_mat.shape
    
    # Initialize the cost matrix
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1, N + 1):
        cost_mat[i, 0] = np.inf
    for i in range(1, M + 1):
        cost_mat[0, i] = np.inf

    # Fill the cost matrix while keeping traceback information
    traceback_mat = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_mat[i, j],      # match (0)
                cost_mat[i, j + 1],  # insertion (1)
                cost_mat[i + 1, j]]  # deletion (2)
            i_penalty = np.argmin(penalty)
            cost_mat[i + 1, j + 1] = dist_mat[i, j] + penalty[i_penalty]
            traceback_mat[i, j] = i_penalty

    # Traceback from bottom right
    i = N - 1
    j = M - 1
    path = [(i, j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i, j]
        if tb_type == 0:
            # Match
            i = i - 1
            j = j - 1
        elif tb_type == 1:
            # Insertion
            i = i - 1
        elif tb_type == 2:
            # Deletion
            j = j - 1
        path.append((i, j))

    cost_mat = cost_mat[1:, 1:]
    return (path[::-1], cost_mat)



In [55]:
#calculate edges
Edges_form=[]
Edges_to=[]
beha_1=[]
beha_2=[]
coor_link=[]
for i in range(len(edges_users)):
    for j in range(len(edges_users[i])):
        for k in range(j+1,len(edges_users[i])):
            user1=edges_users[i][j]
            user2=edges_users[i][k]
            b1=users_behaviour[i][j]
            b2=users_behaviour[i][k]
            l=coordinated_link[i]

            
            if user1<user2:
                Edges_form.append(user1)
                Edges_to.append(user2)
                beha_1.append(b1)
                beha_2.append(b2)
            else:
                Edges_form.append(user2)
                Edges_to.append(user1)
                beha_1.append(b2)
                beha_2.append(b1)
            coor_link.append(l)



df_behaviour=pd.DataFrame()
df_behaviour['From']=Edges_form
df_behaviour['To']=Edges_to
df_behaviour['Beha_1']=beha_1
df_behaviour['Beha_2']=beha_2
df_behaviour['Numeric_LID']=coor_link
del beha_1,beha_2,Edges_form,Edges_to





In [56]:
#Chunk function
def unequal_chunks(list, chunk_size):
    chunks = []
    b=0
    for i in range(0, len(list), chunk_size):
        s=b+chunk_size
        tb=b+chunk_size+1
        while s<len(list) and list[s][0]==list[s-1][0]  and list[s][1]==list[s-1][1]:
            s+=1
            tb+=1
        
        if len(list[b:s]>0):
            chunks.append(list[b:s])
        b=s
    return chunks

In [57]:
#Function to calculate edges for each chunk


def calculated_edges(chunks,df_behaviour,Link_ID):
    second_Edges_form=[]
    second_Edges_to=[]
    second_Edges_weight=[]
    second_beha_1=[]
    second_beha_2=[]
    second_coor_link=[]
    

    counter=0
    TmpFrom= chunks[:,0]
    TmpTo= chunks[:,1]
    mask=np.zeros(len(df_behaviour),dtype=bool)
    preFrom=-1
    preTo=-1
    MEBgraph_temp=MEBgraph.loc[(MEBgraph.Numeric_UID.isin(chunks[:,0]))|(MEBgraph.Numeric_UID.isin(chunks[:,1]))&(MEBgraph.Numeric_LID.isin(chunks[:,4]))]
    for i in range(len(TmpFrom)):
        counter+=1
        v1=TmpFrom[i]
        v2=TmpTo[i]
        if v1!=preFrom or v2!=preTo:
            preFrom=v1
            preTo=v2
            
            mask=(df_behaviour['From']==v1)&(df_behaviour['To']==v2)    
            matrix=df_behaviour[mask][['From','To','Beha_1','Beha_2','Numeric_LID']].to_numpy()
            if len(matrix)>1:
                ZXZX=[] 
                for row in matrix:

                    X=MEBgraph_temp.loc[(MEBgraph_temp.Numeric_UID==row[0])&(MEBgraph_temp.Numeric_LID==row[4])]['TimeDecay']
                    Y=MEBgraph_temp.loc[(MEBgraph_temp.Numeric_UID==row[1])&(MEBgraph_temp.Numeric_LID==row[4])]['TimeDecay']
                    x=np.array(sorted(X))
                    y=np.array(sorted(Y))

                    
                    N = x.shape[0]
                    M = y.shape[0]
                    dist_mat = np.zeros((N, M))
                    for i in range(N):
                        for j in range(M):
                            dist_mat[i, j] = abs(x[i] - y[j])
                    path, cost_mat = dp(dist_mat)
                    
                    
                    ZXZX.append(float(1/(1+(cost_mat[N - 1, M - 1]))))
                added=np.concatenate([np.abs(matrix[:,2:3]-matrix[:,3:4])],axis=1)
                matrix=np.concatenate([matrix,added],axis=1)

                added2= np.array(ZXZX)
                matrix = np.hstack((matrix, added2[:, np.newaxis]))

                sorted_indices=np.argsort(matrix[:,5])
                matrix=matrix[sorted_indices]
                flag=True
                pivot=2

                while flag:
                    s1=matrix[:pivot,2]
                    s2=matrix[:pivot,3]
                    d1=matrix[:pivot,5]

                    cosine=Cosine_similarity(s1,s2)
                    diver=np.sum(d1*d1)
                    if diver==0:
                        diver=0.00000001
                    cosine=cosine/diver
                    if cosine<1 or pivot==len(matrix):
                        flag=False
                        if cosine<1:
                            pivot-=1
                    else:
                        pivot+=1

                if(pivot>1):
                    u1=np.min([matrix[0,0],matrix[0,1]])
                    u2=np.max([matrix[0,0],matrix[0,1]])
                        
                    for ctr in range(pivot):
                        second_Edges_form.append(u1)
                        second_Edges_to.append(u2)
                        second_beha_1.append(matrix[ctr][2])
                        second_beha_2.append(matrix[ctr][3])
                        second_coor_link.append(matrix[ctr][4])
                        second_Edges_weight.append(matrix[ctr][6])


    
    
    del(MEBgraph_temp)
    result = np.zeros((len(second_Edges_form), 6)) 
    result[:,0]=second_Edges_form
    result[:,1]=second_Edges_to
    result[:,2]=second_beha_1
    result[:,3]=second_beha_2
    result[:,4]=second_coor_link
    result[:,5]=second_Edges_weight

    return result

In [58]:
#Multiprocess determing edges function
def multiprocess_edge_calculation(function_reference, file_chunks, arg1,arg2, num_process):
    # Create process pool of pre defined size
    pool = Pool(num_process)

    pbar = tqdm(total=len(file_chunks))

    def update(arg):
        pbar.update()
    
    results = []
    
    for i in range(pbar.total):
        result=pool.apply_async(function_reference, args=(file_chunks[i],)+ (arg1.loc[(arg1.From.isin(file_chunks[i][:,0]))&(arg1.To.isin(file_chunks[i][:,1]))],arg2,), callback=update)
        results.append(result)

    pool.close()
    pool.join()
    
    final_result = np.concatenate([r.get() for r in results], axis=0)
    return final_result

In [59]:
#Decompress list and call function to run
def calculate_edges_with_chunk(dataframe,num_process,chunk_size):
    edge_list=dataframe.values
    chunks = unequal_chunks(edge_list, chunk_size)
    
    Link_ID= np.unique(dataframe['Numeric_LID'])
    Link_ID.sort()
    return multiprocess_edge_calculation(calculated_edges, chunks, dataframe,Link_ID, num_process)

In [60]:
#Main4- calculate edges 
df_behaviour = df_behaviour.sort_values(['From', 'To'])
Result=calculate_edges_with_chunk(df_behaviour,8,1000)
Result_df=pd.DataFrame({'From':Result[:,0],'To':Result[:,1],'Numeric_LID':Result[:,4],'Weight':Result[:,5]})


100%|██████████| 19/19 [-1:59:59<00:00, -17.31it/s]


In [61]:
#creat user_link probability matrix for every users
def user_link_matrix(Posts, user_count, link_count):
    user_links = np.zeros((user_count, link_count))
    user_links[Posts['Numeric_UID'], Posts['Numeric_LID']] = 1
    return user_links

user_link_prob=user_link_matrix(Posts,user_count,link_count)



In [62]:
grouped=Result_df.groupby(['From','To'])

From_ID=[]
To_ID=[]
Weight=[]


for group in grouped:
    matrix=(list(group)[1])[['From','To','Numeric_LID','Weight']].values.tolist()
    temp=0
    for i in range(len(matrix)):
        temp+=matrix[i][3]

    From_ID.append(matrix[0][0])
    To_ID.append(matrix[0][1])    
    Weight.append(temp)




Thirs_df_behaviour=pd.DataFrame({'Source':From_ID,'Target':To_ID,'Weight':Weight})

In [63]:
warnings.filterwarnings("ignore")
def Entropy1(X):
    nonzero_indices = np.nonzero(X)
    nonzero_values = X[nonzero_indices]
    h = np.sum(-nonzero_values * np.log(nonzero_values))
    return h

def Entropy2(X):
    s=sum(X)
    nonzero_indices = np.nonzero(X)
    nonzero_values = X[nonzero_indices]
    h = np.sum(-nonzero_values/s * np.log(nonzero_values/s))
    return h



def JSD_Divergance(user_link1,user_link2):
    XX=np.zeros(len(user_link1))
    s1=sum(user_link1)
    s2=sum(user_link2)
    XX=0.5*(user_link1/s1+user_link2/s2)
    JSD=Entropy1(XX)
    JSD=JSD-0.5*Entropy2(user_link1)
    JSD=JSD-0.5*Entropy2(user_link2)
    return JSD




def divergance_assessment(Thirs_df_behaviour,link_count,user_count,user_link_prob,Result_df):
    warnings.filterwarnings("ignore")
    From_ID=[]
    To_ID=[]
    Numeric_LID=[]
    Weight=[]
    PostIDs_1=[]
    PostIDs_2=[]


    grouped=Result_df.groupby(['From','To'])

    for group in grouped:
            
        new_weight=1      
        matrix=(list(group)[1]).values.tolist()
        node_id1=int(matrix[0][0])
        node_id2=int(matrix[0][1])
        link_IDs = list(np.array(matrix, dtype=object)[:, 2].astype(int))
        
        max_weight1=np.sum(np.array(matrix, dtype=object)[:, 3]) 


        Z=np.zeros(link_count)
        temp_nodes_ids1 = np.where(np.all(user_link_prob[:, link_IDs], axis=1))[0]

        temp_nodes_ids1 = np.delete(temp_nodes_ids1, np.where((temp_nodes_ids1 == node_id1)|(temp_nodes_ids1 == node_id2)))

        len_temp_nodes_ids1=len(temp_nodes_ids1) 

        a0=Thirs_df_behaviour.loc[(Thirs_df_behaviour.Weight.values>=max_weight1)&(((Thirs_df_behaviour.Source.values==node_id1)&(np.isin(Thirs_df_behaviour.Target,temp_nodes_ids1)))|((np.isin(Thirs_df_behaviour.Source,temp_nodes_ids1))&(Thirs_df_behaviour.Target.values==node_id1)))]        
        b0=Thirs_df_behaviour.loc[(Thirs_df_behaviour.Weight.values>=max_weight1)&(((Thirs_df_behaviour.Source.values==node_id2)&(np.isin(Thirs_df_behaviour.Target,temp_nodes_ids1)))|((np.isin(Thirs_df_behaviour.Source,temp_nodes_ids1))&(Thirs_df_behaviour.Target.values==node_id2)))]
        c0 = pd.concat([a0, b0], ignore_index=True)
        Removed_node_IDs=list(set(c0.Source.unique()).union(set(c0.Target.unique())))
        temp_nodes_ids1 = np.delete(temp_nodes_ids1, np.where(np.isin(temp_nodes_ids1, Removed_node_IDs)))


        a=Thirs_df_behaviour.loc[((Thirs_df_behaviour.Source.values==node_id1)&(np.isin(Thirs_df_behaviour.Target,temp_nodes_ids1)))|((np.isin(Thirs_df_behaviour.Source,temp_nodes_ids1))&(Thirs_df_behaviour.Target.values==node_id1))]
        b=Thirs_df_behaviour.loc[((Thirs_df_behaviour.Source.values==node_id2)&(np.isin(Thirs_df_behaviour.Target,temp_nodes_ids1)))|((np.isin(Thirs_df_behaviour.Source,temp_nodes_ids1))&(Thirs_df_behaviour.Target.values==node_id2))]
        c = pd.concat([a, b], ignore_index=True)

        f1=set(c.Source.unique())
        f2=set(c.Target.unique())

        temp_nodes_weights_ids1 =list(f1.union(f2))

        Weight_vector=np.zeros(user_count)

        sum_weight=0
        if (len(temp_nodes_ids1)>0):
            for ctr2 in temp_nodes_ids1:
                X=0
                if ctr2 in temp_nodes_weights_ids1:
                    c1=c.loc[((np.isin(c.Source,[node_id1,node_id2]))&(c.Target.values==ctr2))|((c.Source.values==ctr2)&(np.isin(c.Target,[node_id1,node_id2])))]
                    X=c1['Weight'].max()

                X=(max_weight1-X)/max_weight1
                Weight_vector[ctr2]=X
                sum_weight+=X

            User_IDs=set(temp_nodes_ids1)
            R_temp=(np.multiply(user_link_prob[i, :],Weight_vector[i]) for i in User_IDs)
            Z=np.sum(R_temp, axis=0)
            Z=np.divide(Z,sum_weight)
            JSD1=JSD_Divergance(user_link_prob[node_id1],Z)
            JSD2=JSD_Divergance(user_link_prob[node_id2],Z)
            JSD3=JSD_Divergance(user_link_prob[node_id1],user_link_prob[node_id2])     
            new_weight=(np.subtract(np.min([JSD1,JSD2]),JSD3))
            
        for i in range(len(matrix)):
            From_ID.append(node_id1)
            To_ID.append(node_id2)
            Numeric_LID.append(matrix[i][2])
            x=matrix[i][3]
            if(len_temp_nodes_ids1>0):
                divided = sum_weight/len_temp_nodes_ids1
                x=x*(1-(divided))+x*new_weight*(divided)
            Weight.append(x)

            
    Result_df=pd.DataFrame({'From':From_ID,'To':To_ID,'Numeric_LID':Numeric_LID,'Weight':Weight})
    Result_df=Result_df.loc[Result_df.Weight.values>0]
    return Result_df

Result_df=divergance_assessment(Thirs_df_behaviour,link_count,user_count,user_link_prob,Result_df)


In [ ]:
# Save the final results in a CSV file

distinct_pairs = Posts[['Link', 'Numeric_LID']].drop_duplicates()
merged = pd.merge(Result_df, distinct_pairs, on='Numeric_LID', how='left')
merged=merged.drop(columns=['Numeric_LID'])

distinct_pairs = Posts[['UserID', 'Numeric_UID']].drop_duplicates()
merged = pd.merge(merged, distinct_pairs, left_on='From',right_on='Numeric_UID', how='left')
merged=merged.drop(columns=['From','Numeric_UID'])
merged=merged.rename(columns={'UserID':'Source'})


distinct_pairs = Posts[['UserID', 'Numeric_UID']].drop_duplicates()
merged = pd.merge(merged, distinct_pairs, left_on='To',right_on='Numeric_UID', how='left')
merged=merged.drop(columns=['To','Numeric_UID'])
merged=merged.rename(columns={'UserID':'Target'})
merged=merged.rename(columns={'Link':'Hashtag'})

grouped_df = merged.groupby(['Source', 'Target'])['Weight'].sum().reset_index()

grouped_df.to_csv(main_path_output, encoding='utf-8',index=False)